In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
model_name = "falcon-40b"
model_path = f"/gpfs/user/jsh/{model_name}"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
memories = [torch.cuda.memory_reserved(i) / 1024 ** 3 for i in range(8)]
print(f"{sum(memories)} GiB")

77.9375 GiB


In [9]:
prompt = ["What is deep learning?"]
inputs = tokenizer(prompt, return_tensors="pt").input_ids.cuda(7)
inputs

tensor([[1562,  304, 2845, 2860,   42]], device='cuda:7')

In [7]:
for max_token in [1024, 2048, 4096]:
    total_memory = 0
    outputs = model.generate(
        inputs,
        do_sample=True,
        top_k=50,
        top_p=0.92,
        # min_length=10,
        # max_length=50,
        temperature=0.9,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        max_new_tokens=max_token,
    )
    sentences = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    memories = [torch.cuda.memory_reserved(i) / 1024 ** 3 for i in range(8)]
    print(f"[{max_token}] {sum(memories)} GiB")
    torch.cuda.empty_cache()

/home/jsh/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[1024] 80.98046875 GiB


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[2048] 85.654296875 GiB
[4096] 307.6796875 GiB
